In [42]:
# function to create population based on the population, distribution, and threshold
def create_pop(n,distribution,threshold):
    # create an empty population array
    population = []
    
    # if we have a uniform distribution:
    if distribution == 0:
        # select a random number between 0 and n, and that is the threshold for this individual, added to the population array
        for i in range(0,n):
            population.append(random.randint(0,n))
    
    # if we have a normal distribution:
    if distribution == 1:
        # the population is a random distribution with a mean of n/2, standard deviation of n/5, and size n
        population = numpy.random.normal(n/2, n/5, n)

        # convert the population array from a decimal into an integer
        for i in range(0,n):
            population[i] = int(population[i])

   # if we have a mixed/bimodal distribution:
    if distribution == 2:
        # setting the two means for the two distributions at the first and third quartiles of the distributions
        mu1, sigma1 = 0.25*n,n/5
        mu2, sigma2 = 0.75*n,n/5
        
        # creating two normal distributions, the second adds the remainder of n/2, to account for cases where n is odd
        norm1 = numpy.random.normal(mu1,sigma1,int(n/2))
        norm2 = numpy.random.normal(mu2,sigma2,int(n/2)+(n%2))
        
        # combine the two normal distributions to become the mixed distribution
        population = numpy.concatenate([norm1,norm2])
        
        # convert the population array from a decimal into an integer
        for i in range(0,n):
            population[i] = int(population[i])
    
    # sort the population in increasing order of thresholds, and return
    population = sorted(population)
    return population

In [43]:
# function to iterate through the population and determine how many people will revolt
def revolt(population, thresh, n):
    # creating a time variable to store the number of iterations it takes to run
    time = 0
    # creating a variable to store the count of total people revolting
    count = 0
    
    while time < n:
        # if the person's threshold is less than how many people are currently revolting, increment count by 1
        if population[time] <= count+thresh:
            count += 1
            time += 1
        # if the person's threshold number has not been met, then the revolution stops
        else:
            break
    
    # return the population, count of people revolted, and number of iterations completed
    return [population,count,time]

In [44]:
# function to iterate through the population and determine how many people will revolt, using a behavioral change
# behavioral change: we included the possibility (1% chance) that one person can singlehandedly influence n/10 others
# and a 5% chance that one person can singlehandedly influence n/20 others

def revolt_influential(population, thresh, n):
    # creating a time variable to store the number of iterations it takes to run
    time = 0
    # creating a variable to store the count of total people revolting
    count = 0
    
    while time < n:
        # create a random variable to determine if this person is a revolutionary/has a lot of influence
        influential = random.randint(0,100)

        # if the person's threshold is less than how many people are currently revolting
        if population[time] <= count+thresh:
            # if the person is a revolutionary (1 in 100 chance), add a high number to the count
            if(influential < 15):
                count += int(n/20)
            # if the person is relatively influential, but not a revolutionary, add a moderately high number to the count
            elif(influential < 25):
                count += int(n/50)
            # if the person is not influential, just increment the count by 1
            else:
                count += 1
            time +=1

        # if the person's threshold number has not been met, then the revolution stops
        else:
            break

    # return the population, count of people revolted, and number of iterations completed
    return [population,count,time]

In [48]:
# import all the necessary packages
import random, numpy, pandas
from sklearn.linear_model import LinearRegression

# create tables to store and return the values
df = pandas.DataFrame(columns = ['N',
                                   'Threshold',
                                   'Uniform Dummy',
                                   'Normal Dummy',
                                   'Count/N',
                                   'Time'])

influential_df = pandas.DataFrame(columns = ['N',
                                   'Threshold',
                                   'Uniform Dummy',
                                   'Normal Dummy',
                                   'Count/N',
                                   'Time'])

# iterate through 1000 Monte Carlo simulations
for i in range(0,1000):
    # select a random int value for n between 100 and 1000
    n = random.randint(100,1000)
    # select a random int value for the threshold between 0 and n/50
    thresh = random.randint(int(n/50),int(n/10))
    # select a random number between 0 and 2 to represent the distribution
    dist_select = random.randint(0,2)
    
    # create dummy variables to represent uniform and normal distributions
    uni_dummy = 0
    normal_dummy = 0
    
    # if the random selected variable is 0, then we selected a uniform distribution
    if(dist_select == 0):
        uni_dummy = 1
        normal_dummy = 0
    # if the random selected variable is 1, then we selected a normal distribution
    elif(dist_select == 1):
        uni_dummy = 0
        normal_dummy = 1
    # if the random selected variable is not 0 or 1 (its 2), then we selected a mixed distribution
    else:
        uni_dummy = 0
        normal_dummy = 0
    
    # create the population using the random n, distribution, and threshold values
    population = create_pop(n,dist_select,thresh)
    # iterate through the revolt functions using the population, threshold, and n values
    revolt_values = revolt(population,thresh,n)
    revolt_influential_values = revolt_influential(population,thresh,n)
    
    # set the count values to the second values returned by the revolt functions
    count = revolt_values[1]
    count_influential = revolt_influential_values[1]
    
    # set the time values to the third value returned by the revolt functions
    time = revolt_values[2]
    time_influential = revolt_influential_values[2]
        
    # add a row to the dataframe containing values of n, threshold, uniform dummy, normal dummy, count/n, and time
    df = df.append({'N':n,
                        'Threshold':thresh,
                        'Uniform Dummy':uni_dummy,
                        'Normal Dummy':normal_dummy,
                        'Count/N':count/n,
                        'Time':time},
                       ignore_index=True)
    
    influential_df = influential_df.append({'N':n,
                        'Threshold':thresh,
                        'Uniform Dummy':uni_dummy,
                        'Normal Dummy':normal_dummy,
                        'Count/N':count_influential/n,
                        'Time':time_influential},
                       ignore_index=True)

# print dataframe to csv
df.to_csv("df.csv")
influential_df.to_csv("influential_df.csv")

In [49]:
dataframe = pandas.read_csv("df.csv")

LR = LinearRegression()

x = dataframe[['Threshold', "N", 'Uniform Dummy', "Normal Dummy"]]

y = dataframe[["Count/N"]]

LR.fit(x, y)
print(LR.score(x,y))
print(LR.coef_)
print(LR.intercept_)

0.8516109249128173
[[ 3.84263646e-03 -1.06064457e-04  7.77672343e-03 -8.68407549e-01]]
[0.82056478]


In [50]:
dataframe_influential = pandas.read_csv("influential_df.csv")

LR_influential = LinearRegression()

x_influential = dataframe_influential[['Threshold', "N", 'Uniform Dummy', "Normal Dummy"]]

y_influential = dataframe_influential[["Count/N"]]

LR.fit(x_influential, y_influential)
print(LR.score(x_influential,y_influential))
print(LR.coef_)
print(LR.intercept_)

0.8382761066701765
[[ 0.01280023  0.0092303  -0.01596349 -1.06793976]]
[0.29696201]
